In [ ]:
import pandas as pd
from neo4j import GraphDatabase

# Neo4j connection details
URI = "neo4j://38.242.232.192:7687"
AUTH = ("nest", "neurons-newbie")

# CSV file path
csv_file_path = "/content/GoodCSV (1).csv"  # Update with your actual file path

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Clean the data (optional, remove rows with NaN in required columns)
df = df.dropna(subset=["Subject", "Relationship", "Object"])

# Function to create nodes and undirected relationships in Neo4j
def create_nodes_and_relationships(tx, subject, relationship, obj):
    # Create Cypher query with distinct labels
    query = """
    MERGE (subject:SubjectNode {name: $subject})  
    MERGE (object:ObjectNode {name: $object})   
    MERGE (subject)-[:%s]-(object)               
    """ % relationship.upper()  # Relationship types should be in uppercase

    # Execute query
    tx.run(query, subject=subject, object=obj)

# Connect to Neo4j and process the data
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            for index, row in df.iterrows():
                subject = row["Subject"]
                relationship = row["Relationship"]
                obj = row["Object"]

                session.write_transaction(create_nodes_and_relationships, subject, relationship, obj)
                print(f"Processed row {index}: {subject} -[{relationship}]- {obj}")

    print("Data import complete.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
import pandas as pd
import ast
from neo4j import GraphDatabase

# Neo4j connection details
URI = "neo4j://38.242.232.192:7687"
AUTH = ("nest", "neurons-newbie")

# Function to check if a node exists in Neo4j
def check_node_exists(tx, node_name, label):
    query = f"""
    MATCH (node:{label} {{name: $node_name}})
    RETURN COUNT(node) > 0 AS node_exists
    """
    result = tx.run(query, node_name=node_name).single()
    return result["node_exists"]

# Function to delete a similar object node and repoint its relationships
def delete_and_repoint(tx, similar_object, main_object):
    query = """
    MATCH (similar:ObjectNode {name: $similar_object})-[r]-(other)
    WITH similar, r, other
    MATCH (main:ObjectNode {name: $main_object})
    CALL {
        WITH r
        RETURN type(r) AS relationship_type
    }
    MERGE (main)-[newRel:`${relationship_type}`]-(other)
    DELETE r, similar
    """
    tx.run(query, similar_object=similar_object, main_object=main_object)

# Load the CSV file containing similar objects
csv_file_path = "/content/filtered_results_with_similars (2).csv"  # Update with the correct path to your CSV file
df = pd.read_csv(csv_file_path)

# Connect to Neo4j and process the data
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            for index, row in df.iterrows():
                object_name = row["Object"]
                similar_objects = row["Similar_Objects"]

                # Convert Similar_Objects string to a list safely using ast.literal_eval
                similar_objects_list = ast.literal_eval(similar_objects) if similar_objects else []

                # Check if the main object exists (ensure it is an ObjectNode)
                main_object_exists = session.read_transaction(check_node_exists, object_name, "ObjectNode")

                if not main_object_exists:
                    print(f"The main object '{object_name}' does not exist in the database. Skipping.")
                    continue

                print(f"Processing similar objects for '{object_name}':")

                for similar_object in similar_objects_list:
                    # Check if the similar object exists (ensure it is an ObjectNode)
                    similar_object_exists = session.read_transaction(check_node_exists, similar_object, "ObjectNode")

                    if similar_object_exists:
                        print(f"  - The similar object '{similar_object}' exists. Deleting and repointing relationships.")
                        session.write_transaction(delete_and_repoint, similar_object, object_name)
                    else:
                        print(f"  - The similar object '{similar_object}' does not exist. Skipping.")

except Exception as e:
    print(f"An error occurred: {e}")
